In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import cv2
import os
import numpy as np
import pandas as pd
from time import time
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from scipy import signal

from automatic_meter_reader import AutomaticMeterReader

# Parameters

In [ ]:
camera_model = "espcam_120_deg"
meter_models = {
    "water_meter": "lorenz_1997",
    "cold_water_meter": "wehrle_cold",
    "hot_water_meter": "wehrle_hot",
    "gas_meter": "gas_bk25",
}

meter_name = "water_meter"
meter_model = meter_models[meter_name]

images_dir = os.path.join("raw_images", meter_name)
ur = AutomaticMeterReader(camera_model, meter_model)
input_filenames = [os.path.join(images_dir, fn) for fn in sorted(os.listdir(images_dir)) if "debug" not in fn]  # if fn > "cold_water_meter_20210308_203742.jpg"
print(len(input_filenames))

cur_image_nr = 0

# Find uncertain predictions

In [ ]:
data = []
for i, fn in enumerate(input_filenames):
    if i % 10 != 0:
        continue
    img = cv2.imread(fn)
    ur.readout(img)
    min_confidence_index = np.argmin(ur.confidences[:-1]) # Ignore last digit
    min_confidence = ur.confidences[min_confidence_index]
    print(i, min_confidence_index, min_confidence, ur.measurement, fn)
    
    data.append({
        "filename": fn,
        "min_confidence": min_confidence,
        "measurement": ur.measurement or np.NaN
    })
data = pd.DataFrame(data)

In [ ]:
data["true_measurement"] = signal.medfilt(data["measurement"], 31)
data["diff"] = data["measurement"] - data["true_measurement"]
data

In [ ]:
plt.figure()
#plt.plot(data["measurement"], label="raw")
#plt.plot(data["true_measurement"], label="filtered")
plt.plot(data["diff"].iloc[250:-250], label="diff")

plt.legend()

In [ ]:
# (data["diff"].abs() > 0.01)
df_filtered = data[(data["min_confidence"] < 0.98) & (data.index > 250) & (data.index < len(data) - 250)]
filenames = list(df_filtered["filename"])
df_filtered

# Annotate number of digits

In [ ]:
def redraw():
    for ax in [ax1, ax2]:
        ax.clear()
        ax.axis("off")
    
    fig.suptitle("%s %d/%d (%.3f m3)" % (os.path.basename(filenames[cur_image_nr]), cur_image_nr + 1, len(filenames), ur.measurement or -1))
    if last_img is not None:
        ax1.imshow(cv2.cvtColor(last_img, cv2.COLOR_BGR2RGB))
    ax2.imshow(cv2.cvtColor(ur.img_debug, cv2.COLOR_BGR2RGB))
    
    for i, digit_conf in enumerate(ur.meter_config["register"]["digits"]):
        x0, y0, dx, dy = digit_conf["roi"]
        color = 'r' if i not in active_digits else 'g'
        ax2.add_patch(patches.Rectangle((x0 - 0.5, y0 - 0.5), dx, dy, linewidth=2.0, ls="-", edgecolor=color, alpha=1.0, facecolor='none'))

def load_image():
    global last_img
    global active_digits
    active_digits = []
    last_img = ur.img_debug
    log.append("load image %d %s" % (cur_image_nr, filenames[cur_image_nr]))
    img = cv2.imread(filenames[cur_image_nr])
    ur.readout(img)
    redraw()
    
def annotate():
    log.append("annotate %s: %s" % (filenames[cur_image_nr], active_digits))
    for digit_nr in active_digits:
        digit_img = ur.digit_imgs[digit_nr]
        out_file = os.path.join(out_folder, "%d_%s" % (digit_nr, os.path.basename(filenames[cur_image_nr])))
        log.append("save digit file %s" % (out_file))
        cv2.imwrite(out_file, digit_img)
    
def key_press_event(event):
    global cur_image_nr, active_digits
    log.append("onclick %s" % (event.key))
    if event.key == "right" and cur_image_nr + 1 < len(filenames):
        cur_image_nr += 1
        load_image()
    elif event.key == "left" and cur_image_nr > 0:
        cur_image_nr -= 1
        load_image()
    elif event.key in ["1", "2", "3", "4", "5", "6", "7", "8"]:
        digit_nr = int(event.key) - 1
        if digit_nr in active_digits:
            active_digits.remove(digit_nr)
        else:
            active_digits.append(digit_nr)
        redraw()
    elif event.key == "enter":
        annotate()
        cur_image_nr += 1
        load_image()
    elif event.key == "escape":
        active_digits = []
    log.append("onclick %s done" % (event.key))
    
out_folder ="digits_to_annotate"
if not os.path.isdir(out_folder):
    os.mkdir(out_folder)
active_digits = []
last_img = None
log = []
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
cid = fig.canvas.mpl_connect('key_press_event', key_press_event)
load_image()

In [ ]:
### print("\n".join(log))